# Testing Agents v2 in AI Foundry
This notebook creates and runs an agent from AI Foundry

Best way to set it up is with [uv](https://docs.astral.sh/uv/)

1. in `agents` directory run `uv sync` (if python is missing, install it using `uv python install`)
2. in VSCode press [Ctrl+Shift+P] and select `Python: Select Interpreter`, choose the one from `agents` directory: `./agents/.venv/bin/python3.xx`

## Step 1
Create Project client

In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import atexit
import jsonref
import os
from azure.identity.aio import DefaultAzureCredential, AzureDeveloperCliCredential
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import (
    MCPTool,
    Tool,
)
from openai import AsyncOpenAI
from openai.types.responses.response_input_param import (
    McpApprovalResponse,
    ResponseInputParam,
)
from dotenv import load_dotenv
from src.agents_utils import agents_utils


# Load environment variables from the .env file
load_dotenv(override=True)

print("🚀 Initializing AI Foundry Agent Testing...")
print("=" * 60)

endpoint = os.environ.get("AZURE_AI_FOUNDRY_CONNECTION_STRING")
deployment_name = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", None)
tenant_id = os.environ.get("AZURE_TENANT_ID", None)

print("\n📋 Configuration:")
print(f"   📍 Endpoint: {endpoint[:50]}..." if endpoint else "   ⚠️ Endpoint: Not set")
print(f"   🤖 Model: {deployment_name}" if deployment_name else "   ⚠️ Model: Not set")
print(f"   📦 API Version: {api_version or 'Default'}")

ai_agent_settings = {
    "endpoint": endpoint,
    "model_deployment_name": deployment_name,
    "api_version": api_version,
}

# Setup credentials
print("\n🔐 Setting up authentication...")
if os.environ.get("USE_AZURE_DEV_CLI") == "true":
    print("   ✅ Using Azure Developer CLI Credential")
    creds = AzureDeveloperCliCredential(tenant_id=tenant_id)
else:
    print("   ✅ Using Default Azure Credential")
    creds = DefaultAzureCredential()

await creds.__aenter__()
print("   🔓 Credentials initialized successfully!")

# Initialize clients
print("\n🔧 Initializing clients...")
client = AIProjectClient(endpoint=endpoint, credential=creds)
await client.__aenter__()
agents_client = agents_utils(client)
print("   ✅ AI Project Client ready")
print("   ✅ Agents Client ready")

# List connections
print("\n" + "=" * 60)
print("🔗 AVAILABLE CONNECTIONS")
print("=" * 60)

model_gateway_connection_static = None
model_gateway_connection_dynamic = None
ai_gateway_connection_static = None
ai_gateway_connection_dynamic = None

connection_count = 0
async for connection in client.connections.list():
    connection_count += 1
    icon = (
        "🌐"
        if connection.type == "ModelGateway"
        else "🔌" if connection.type == "ApiManagement" else "📡"
    )
    default_badge = " ⭐ DEFAULT" if connection.is_default else ""
    print(f"\n{icon} {connection.name}{default_badge}")
    print(f"   Type: {connection.type}")
    print(f"   ID: {connection.id[:50]}...")

    if connection.type == "ModelGateway" and "static" in connection.name.lower():
        model_gateway_connection_static = connection.name
        print("   📌 → Assigned to: model_gateway_connection_static")
    if connection.type == "ModelGateway" and "static" not in connection.name.lower():
        model_gateway_connection_dynamic = connection.name
        print("   📌 → Assigned to: model_gateway_connection_dynamic")
    if connection.type == "ApiManagement" and "static" in connection.name.lower():
        ai_gateway_connection_static = connection.name
        print("   📌 → Assigned to: ai_gateway_connection_static")
    if connection.type == "ApiManagement" and "static" not in connection.name.lower():
        ai_gateway_connection_dynamic = connection.name
        print("   📌 → Assigned to: ai_gateway_connection_dynamic")

print(f"\n📊 Total connections found: {connection_count}")

# List agents
print("\n" + "=" * 60)
print("🤖 EXISTING AGENTS")
print("=" * 60)

agents = await agents_client.get_agents()
if agents:
    for agent in agents:
        print(f"\n🤖 {agent.name}")
        print(f"   ID: {agent.id}")
        print(f"   Version: {agent.versions.latest.version}")
else:
    print("\n   ℹ️ No agents found in this project")

# Summary
print("\n" + "=" * 60)
print("✅ SETUP COMPLETE - Connection Summary")
print("=" * 60)
print(
    f"   🌐 Model Gateway (Static):  {'✅ ' + model_gateway_connection_static if model_gateway_connection_static else '❌ Not found'}"
)
print(
    f"   🌐 Model Gateway (Dynamic): {'✅ ' + model_gateway_connection_dynamic if model_gateway_connection_dynamic else '❌ Not found'}"
)
print(
    f"   🔌 AI Gateway (Static):     {'✅ ' + ai_gateway_connection_static if ai_gateway_connection_static else '❌ Not found'}"
)
print(
    f"   🔌 AI Gateway (Dynamic):    {'✅ ' + ai_gateway_connection_dynamic if ai_gateway_connection_dynamic else '❌ Not found'}"
)
print("\n🎉 Ready to run agent tests!")


async def cleanup():
    """Close all async clients properly"""
    try:
        await client.close()
    except Exception:
        await client.__aexit__(None, None, None)
    try:
        await creds.close()
    except Exception:
        await creds.__aexit__(None, None, None)
    print("🧹 Clients closed")


def sync_cleanup():
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            loop.create_task(cleanup())
        else:
            loop.run_until_complete(cleanup())
    except Exception:
        print("⚠️ Cleanup failed")
        pass


atexit.register(sync_cleanup)

## Run Tests

In [ ]:
# Run multiple requests for each gateway connection and capture request IDs
import pandas as pd
from openai import APIStatusError, APIError
from azure.core.exceptions import HttpResponseError

print("🧪 GATEWAY CONNECTION TESTS")
print("=" * 60)
print("Testing each gateway connection with multiple requests to verify")
print("connectivity and capture request IDs for debugging.\n")

# Collect all gateway connections to test
gateway_connections = {
    "model_gateway_static": model_gateway_connection_static,
    "model_gateway_dynamic": model_gateway_connection_dynamic,
    "ai_gateway_static": ai_gateway_connection_static,
    "ai_gateway_dynamic": ai_gateway_connection_dynamic,
}
deployment_model: str = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

# Filter out None connections
active_connections = {k: v for k, v in gateway_connections.items() if v is not None}
print(f"📊 Found {len(active_connections)} active connections to test:")
for name in active_connections.keys():
    print(f"   • {name}")

results = []
num_requests = 20
delete_agent_before_create = False

print(f"\n⚙️ Test Configuration:")
print(f"   • Requests per connection: {num_requests}")
print(f"   • Model: {deployment_model}")
print(f"   • Delete agent before create: {delete_agent_before_create}")

for conn_name, conn_value in active_connections.items():
    print(f"\n{'─' * 60}")
    print(f"🔄 Testing: {conn_name}")
    print(f"   Connection: {conn_value}")
    print(f"{'─' * 60}")

    for i in range(num_requests):
        request_id = None
        result = None
        error_msg = None

        try:
            agent_name = f"TestAgent_{conn_name}".replace(" ", "-").replace("_", "-")[
                :63
            ]

            # Create agent for this connection
            print(f"   📝 Request {i+1}/{num_requests}: Creating agent...", end=" ")
            agent = await agents_client.create_agent(
                name=agent_name,
                model_gateway_connection=conn_value,
                deployment_name=deployment_model,
                delete_before_create=delete_agent_before_create,
            )

            openai_client = client.get_openai_client()

            # Create conversation
            conversation = await openai_client.conversations.create(
                items=[
                    {
                        "type": "message",
                        "role": "user",
                        "content": f"What is 2 + {i}? Reply with just the number.",
                    }
                ],
            )

            # Get response
            response = await openai_client.responses.create(
                conversation=conversation.id,
                extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
                input="",
            )

            # Extract request ID from response
            request_id = response._request_id
            result = "SUCCESS"
            output = response.output_text[:50] if response.output_text else "No output"

            results.append(
                {
                    "Connection": conn_name,
                    "Model": deployment_model,
                    "Request #": i + 1,
                    "Request ID": request_id,
                    "Result": result,
                    "Output": output,
                }
            )
            print(f"✅ SUCCESS")
            print(f"      Response: {output}")
            print(f"      Request ID: {request_id}")

        except (APIStatusError, APIError, HttpResponseError) as e:
            request_id = (
                e.response.headers.get("x-request-id", "N/A")
                if hasattr(e, "response")
                else "N/A"
            )
            result = "ERROR"
            error_msg = str(e)

            results.append(
                {
                    "Connection": conn_name,
                    "Model": deployment_model,
                    "Request #": i + 1,
                    "Request ID": request_id,
                    "Result": result,
                    "Output": error_msg,
                }
            )
            print(f"❌ ERROR")
            print(f"      {error_msg}")
            print(f"      Request ID: {request_id}")

        except Exception as e:
            result = "ERROR"
            error_msg = str(e)

            results.append(
                {
                    "Connection": conn_name,
                    "Model": deployment_model,
                    "Request #": i + 1,
                    "Request ID": "N/A",
                    "Result": result,
                    "Output": error_msg,
                }
            )
            print(f"❌ ERROR")
            print(f"      {error_msg}")

# Create DataFrame and display results
df = pd.DataFrame(results)

print("\n" + "=" * 60)
print("📊 RESULTS TABLE")
print("=" * 60)
print(df.to_string(index=False))

# Save results to CSV
csv_file = "gateway_connection_test_results.csv"
df.to_csv(csv_file, index=False)
print(f"\n💾 Results saved to: {csv_file}")

# Summary statistics
print("\n" + "=" * 60)
print("📈 SUMMARY")
print("=" * 60)
summary = df.groupby(["Connection", "Result"]).size().unstack(fill_value=0)
print(summary)

# Calculate success rate
total = len(results)
successes = len([r for r in results if r["Result"] == "SUCCESS"])
success_rate = (successes / total * 100) if total > 0 else 0
print(f"\n🎯 Overall Success Rate: {successes}/{total} ({success_rate:.1f}%)")

if success_rate == 100:
    print("🎉 All tests passed!")
elif success_rate >= 50:
    print("⚠️ Some tests failed - check the results above")
else:
    print("❌ Most tests failed - investigate connection issues")

## Call Foundry directly

In [ ]:
print("🔗 DIRECT FOUNDRY CALL")
print("=" * 60)
print("Calling AI Foundry directly without agent abstraction")
print(f"   📍 Connection: {model_gateway_connection_static}")
print(f"   🤖 Model: {deployment_name}")
print()

openai_client = client.get_openai_client()

response = await openai_client.responses.create(
    model=f"{model_gateway_connection_static}/{deployment_name}",
    input="I am going to Paris, what should I see? Keep your answer brief.",
    instructions="You are a helpful assistant.",
)

print("✅ Response received!")
print(f"   🆔 Request ID: {response._request_id}")
print("\n💬 Response:")
print("─" * 40)
print(response.output_text)
print("─" * 40)

## Run agent using static gateway

In [ ]:
print("🤖 AGENT WITH STATIC GATEWAY")
print("=" * 60)
print(f"   🔗 Connection: {ai_gateway_connection_static}")
print()

print("📝 Creating agent...")
agent = await agents_client.create_agent(
    name="MyV2Agent",
    model_gateway_connection=ai_gateway_connection_static,
    delete_before_create=True,
)
print(f"   ✅ Agent created: {agent.name}")

openai_client = client.get_openai_client()
conversation = await openai_client.conversations.create(
    items=[
        {
            "type": "message",
            "role": "user",
            "content": "What is the size of Poland in square miles?",
        }
    ],
)
print(f"   💬 Conversation started: {conversation.id[:20]}...")

print("\n⏳ Waiting for response...")
response = await openai_client.responses.create(
    conversation=conversation.id,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    input="",
)

print("\n✅ Response received!")
print(f"   🆔 Response ID: {response.id}")
print("\n💬 Output:")
print("─" * 40)
print(response.output_text)
print("─" * 40)
print(f"\n💰 Usage: {response.to_dict()['usage']}")

## Run agent using dynamic gateway

In [ ]:
print("🤖 AGENT WITH DYNAMIC GATEWAY")
print("=" * 60)
print(f"   🔗 Connection: {model_gateway_connection_dynamic}")
print()

print("📝 Creating agent...")
agent = await agents_client.create_agent(
    name="MyV2Agent",
    model_gateway_connection=model_gateway_connection_dynamic,
    delete_before_create=True,
)
print(f"   ✅ Agent created: {agent.name}")

openai_client = client.get_openai_client()
conversation = await openai_client.conversations.create(
    items=[
        {
            "type": "message",
            "role": "user",
            "content": "What is the history of Warsaw? Keep it brief.",
        }
    ],
)
print(f"   💬 Conversation started: {conversation.id[:20]}...")

print("\n⏳ Waiting for response...")
response = await openai_client.responses.create(
    conversation=conversation.id,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    input="",
)

print("\n✅ Response received!")
print("\n💬 Output:")
print("─" * 40)
print(response.output_text)
print("─" * 40)
print(f"\n💰 Usage: {response.to_dict()['usage']}")

## Streaming

In [ ]:
print("🌊 STREAMING RESPONSE TEST")
print("=" * 60)
print("Testing real-time streaming of agent responses")
print()

print("📝 Creating agent...")
agent = await agents_client.create_agent(
    name="MyAgentGpt5Mini",
    model_gateway_connection=model_gateway_connection_dynamic,
    delete_before_create=True,
)
print(f"   ✅ Agent: {agent.name}")

openai_client = client.get_openai_client()
conversation = await openai_client.conversations.create(
    items=[
        {
            "type": "message",
            "role": "user",
            "content": "Tell me hi in 10 random languages.",
        }
    ],
)
print(f"   💬 Conversation: {conversation.id[:20]}...")

# Create streaming response
response_stream_events = await openai_client.responses.create(
    conversation=conversation.id,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    input="",
    stream=True,
)

print("\n🌊 Streaming response:")
print("─" * 40)
events_received = []

async for event in response_stream_events:
    previous_event = events_received[-1] if len(events_received) > 0 else None
    if previous_event and previous_event["type"] == event.type:
        previous_event["count"] += 1
    else:
        events_received.append({"type": event.type, "count": 1})

    if event.type == "response.created":
        print(f"🆕 Stream started (ID: {event.response.id})\n")
    elif event.type == "response.output_text.delta":
        print(event.delta, end="", flush=True)
    elif event.type == "response.text.done":
        print("\n" + "─" * 40)
        print("✅ Text complete")
    elif event.type == "response.completed":
        print(f"\n🎉 Response completed!")
        print(f"💰 Usage: {event.response.to_dict()['usage']}")

print("\n📊 Events received:")
for index, event in enumerate(events_received):
    print(f"   {index+1}. {event['type']} (×{event['count']})")

## Tool Calls

In [ ]:
from src.agents_utils import process_stream
from openai import NOT_GIVEN

print("🔧 STREAMING MCP TOOL CALLS")
print("=" * 60)
print("Testing MCP tool integration with streaming responses")
print()

# Configure MCP tool
mcp_tool = MCPTool(
    server_label="ms-learn",
    server_url="https://learn.microsoft.com/api/mcp",
    require_approval="always",
)
print("🔌 MCP Tool configured:")
print(f"   Label: {mcp_tool.server_label}")
print(f"   URL: {mcp_tool.server_url}")
print(f"   Approval: {mcp_tool.require_approval}")

tools: list[Tool] = [mcp_tool]

print("\n📝 Creating agent with tools...")
agent = await agents_client.create_agent(
    name="MyAgentGpt5MiniTools",
    model_gateway_connection=model_gateway_connection_static,
    delete_before_create=True,
    instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
    tools=tools,
)
print(f"   ✅ Agent: {agent.name}")
print(f"   🤖 Model: {agent.versions.latest.definition.model}")

openai_client = client.get_openai_client()
conversation = await openai_client.conversations.create(
    items=[
        {
            "type": "message",
            "role": "user",
            "content": "Find me learning paths on Microsoft Learn about Azure AI services.",
        }
    ],
)
print(f"   💬 Conversation: {conversation.id[:20]}...")

# Streaming loop with tool approval
events_received = []
input_list = []
response_id = None
input = ""
request_count = 0

print("\n🔄 Starting tool call loop...")
print("─" * 60)

while True:
    response_stream_events = await openai_client.responses.create(
        conversation=conversation.id if response_id is None else NOT_GIVEN,
        previous_response_id=response_id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        input=input,
        stream=True,
    )

    print(f"\n🌊 Stream iteration #{request_count + 1}:")
    events_received, input_list, response_id, full_response = await process_stream(
        response_stream_events
    )

    print(f"\n📊 Events in this iteration:")
    for index, event in enumerate(events_received):
        print(f"   {index+1}. {event['type']} (×{event['count']})")

    if len(input_list) == 0:
        break

    input = input_list
    print(f"\n✅ Auto-approving {len(input_list)} tool request(s)...")
    request_count += 1

print("\n" + "=" * 60)
print("🎉 Tool call loop completed!")
print(f"   Total iterations: {request_count + 1}")

## Non Streaming MCP

In [ ]:
from src.agents_utils import process_response

print("🔧 NON-STREAMING MCP TOOL CALLS")
print("=" * 60)
print("Testing MCP tools without streaming (synchronous)")
print()

# Configure MCP tool
mcp_tool = MCPTool(
    server_label="docs",
    server_url="https://gitmcp.io/Azure/azure-rest-api-specs",
    require_approval="always",
)
print(f"🔌 MCP Tool configured:")
print(f"   Label: {mcp_tool.server_label}")
print(f"   URL: {mcp_tool.server_url}")

# Load OpenAPI spec (optional - not used in this example)
with open("weather.json", "r") as f:
    openapi_weather = jsonref.loads(f.read())
    openapi_weather["servers"] = [{"url": "https://wttr.in"}]
print(f"🌤️ Weather OpenAPI spec loaded (available if needed)")

tools: list[Tool] = [mcp_tool]

print("\n📝 Creating agent with tools...")
agent = await agents_client.create_agent(
    name="MyAgentGpt5MiniTools",
    model_gateway_connection=model_gateway_connection_static,
    delete_before_create=True,
    instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
    tools=tools,
)
print(f"   ✅ Agent: {agent.name}")

openai_client = client.get_openai_client()
response_id = None
input = ""
input_list: ResponseInputParam = []
request_count = 0
conversation = await openai_client.conversations.create(
    items=[
        {"type": "message", "role": "user", "content": "Summarize the readme from attached MCP tool for me. Keep it very brief."}
    ],
)
print(f"   💬 Conversation: {conversation.id[:20]}...")

print("\n⏳ Sending initial request...")
while True:
    response = await openai_client.responses.create(
        conversation=conversation.id if response_id is None else NOT_GIVEN,
        previous_response_id=response_id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        input=input,
        stream=False,
    )

    print(f"\n🌊 Stream iteration #{request_count + 1}:")
    events_received, input_list, response_id, full_response = await process_response(
        response
    )

    print("\n📊 Events in this iteration:")
    for index, event in enumerate(events_received):
        print(f"   {index+1}. {event['type']} (×{event['count']})")

    if len(input_list) == 0:
        break

    input = input_list
    print(f"\n✅ Auto-approving {len(input_list)} tool request(s)...")
    request_count += 1

response = await openai_client.responses.create(
    conversation=conversation.id,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    input="",
)

print("\n" + "=" * 60)
print("✅ RESULT")
print("=" * 60)
print(response.output_text)
print("\n💰 Full response details:")
print(f"   Usage: {response.to_dict().get('usage', 'N/A')}")

## 🔧 Agent MCP Loop Tests
Run multiple requests using an agent with MCP tools and capture:
- ✅ Success/Error status for each iteration
- 🆔 Request IDs for debugging
- 📊 Summary statistics and success rate
- 💾 Results exported to CSV

In [ ]:
import pandas as pd
from openai import APIStatusError, APIError, NOT_GIVEN
from azure.core.exceptions import HttpResponseError
from src.agents_utils import process_response


print("🔧 MCP WITH AGENT - LOOP TEST")
print("=" * 60)
print("Running MCP tools with agent abstraction")
print("Testing with 10 iterations to capture errors and request IDs")
print()

# Configure MCP tool
mcp_tool = MCPTool(
    server_label="weather",
    server_url="https://aca-mcp-qczp34j2qg7pk.ashyocean-7ea49412.westus.azurecontainerapps.io/mcp/mcp",
    require_approval="never",
)
print(f"🔌 MCP Tool: {mcp_tool.server_label}")
print(f"   URL: {mcp_tool.server_url}")

tools: list[Tool] = [mcp_tool]

# Create agent once at the top
print("\n📝 Creating agent...")
agent = await agents_client.create_agent(
    name="MCPLoopTestAgent",
    # model_gateway_connection=model_gateway_connection_static,
    deployment_name="gpt-5-mini",
    delete_before_create=True,
    instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
    tools=tools,
)
print(f"   ✅ Agent: {agent.name}")
print(f"   🤖 Model: {agent.versions.latest.definition.model}")

openai_client = client.get_openai_client()

# Configuration
num_iterations = 10
results = []

print(f"\n⚙️ Test Configuration:")
print(f"   • Iterations: {num_iterations}")
print(f"   • Agent: {agent.name}")
print()

for iteration in range(num_iterations):
    print(f"\n{'─' * 60}")
    print(f"🔄 Iteration {iteration + 1}/{num_iterations}")
    print(f"{'─' * 60}")
    
    request_id = None
    result_status = None
    error_msg = None
    output_text = None
    
    try:
        # Create new conversation for each iteration
        random_number_a = iteration * 7 + 3
        random_number_b = iteration * 11 + 5
        conversation = await openai_client.conversations.create(
            items=[
                {"type": "message", "role": "user", "content": f"Add {random_number_a} and {random_number_b} using MCP tool."}
            ],
        )
        print(f"   💬 Conversation: {conversation.id[:20]}...")
        response_id = None
        input = ""
        input_list: ResponseInputParam = []
        request_count = 0

        while True:
            print("   ⏳ Sending request...")
            response = await openai_client.responses.create(
                conversation=conversation.id if response_id is None else NOT_GIVEN,
                previous_response_id=response_id,
                extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
                input=input,
            )

            print(f"\n🌊 Request iteration #{request_count + 1}:")
            events_received, input_list, response_id, full_response = await process_response(
                response
            )

            if len(input_list) == 0:
                break

            input = input_list
            print(f"\n✅ Auto-approving {len(input_list)} tool request(s)...")
            request_count += 1


        request_id = response._request_id

        result_status = "SUCCESS"
        output_text = response.output_text[:100] if response.output_text else "No output"
        print(f"   ✅ SUCCESS")
        print(f"      Request ID: {request_id}")
        print(f"      Output: {output_text[:50]}...")

    except (APIStatusError, APIError, HttpResponseError) as e:
        request_id = (
            e.response.headers.get("x-request-id", "N/A")
            if hasattr(e, "response") and e.response is not None
            else "N/A"
        )
        result_status = "ERROR"
        error_msg = str(e)
        print(f"   ❌ ERROR")
        print(f"      Request ID: {request_id}")
        print(f"      Error: {error_msg}")

    except Exception as e:
        result_status = "ERROR"
        error_msg = str(e)
        print(f"   ❌ ERROR")
        print(f"      Request ID: N/A")
        print(f"      Error: {error_msg}")

    # Record results
    results.append({
        "Iteration": iteration + 1,
        "Request ID": request_id or "N/A",
        "Result": result_status,
        "Output/Error": output_text if result_status == "SUCCESS" else error_msg,
    })

# Create DataFrame and display results
df_results = pd.DataFrame(results)

print("\n" + "=" * 60)
print("📊 RESULTS TABLE")
print("=" * 60)
print(df_results.to_string(index=False))

# Save results to CSV
csv_file = "mcp_agent_test_results.csv"
df_results.to_csv(csv_file, index=False)
print(f"\n💾 Results saved to: {csv_file}")

# Summary statistics
print("\n" + "=" * 60)
print("📈 SUMMARY")
print("=" * 60)
total = len(results)
successes = len([r for r in results if r["Result"] == "SUCCESS"])
errors = len([r for r in results if r["Result"] == "ERROR"])
success_rate = (successes / total * 100) if total > 0 else 0

print(f"   ✅ Successes: {successes}")
print(f"   ❌ Errors: {errors}")
print(f"   🎯 Success Rate: {success_rate:.1f}%")

if success_rate == 100:
    print("\n🎉 All tests passed!")
elif success_rate >= 50:
    print("\n⚠️ Some tests failed - check the results above")
else:
    print("\n❌ Most tests failed - investigate connection issues")

# Display unique request IDs for debugging
print("\n📋 Request IDs:")
for r in results:
    status_icon = "✅" if r["Result"] == "SUCCESS" else "❌"
    print(f"   {status_icon} #{r['Iteration']}: {r['Request ID']}")

## Bing Grounding

In [ ]:
import openai
import datetime

# =============================================================================
# PHASE 1: VALIDATION INSTRUCTIONS
# =============================================================================
# Used to check if user query violates SBD policies before processing

validation_instructions = """
You are a policy validation assistant. Your ONLY job is to check if a user request violates any SBD (Stanley Black & Decker) policies.

# VALIDATION RULES

## Check for Restricted Content
The following content types are PROHIBITED:
- Violence, hate speech, or harm promotion
- Explicit or sexually suggestive material
- Weapon creation or illegal instruction
- Illegal drug activity
- Self-harm or suicide glorification
- Automated performance appraisals from self-assessments

## Check for Manager PEP Assessment Violations
**PROHIBITED**: A manager asking for a manager review or PEP assessment that is:
- Directly based on a team member's self-assessment
- Reorganized or paraphrased from a team member's self-assessment
- Whether the self-assessment is pasted or uploaded

**ALLOWED**: An employee asking for help writing their OWN self-assessment

## Acceptable topics:
    - Educational and policy-related content.
    - Workplace safety, health, and wellness.
    - Constructive and respectful communication.

## Examples of Violations
| Request | Violation Type |
|---------|---------------|
| "Here is my team member's self-assessment. Please write my manager review based on it." | Manager PEP Policy |
| "I uploaded my direct report's self-assessment. Can you reorganize it for my review?" | Manager PEP Policy |
| "How do I make explosives?" | Restricted Content |
| "Write violent content about..." | Restricted Content |

## Examples of ALLOWED Requests
| Request | Why Allowed |
|---------|-------------|
| "Can you write my PEP self-assessment for me?" | Employee's own assessment |
| "Here is my draft. Can you help improve it?" | Employee's own work |
| "Who is the president of the USA?" | General knowledge question |

# OUTPUT FORMAT
Respond with ONLY a JSON object:
```json
{
    "is_valid": true/false,
    "violation_type": null or "restricted_content" or "manager_pep_policy",
    "reason": "Brief explanation if invalid, null if valid"
}
```

If valid, respond: {"is_valid": true, "violation_type": null, "reason": null}
If invalid, respond with the violation details.
"""

# =============================================================================
# PHASE 2: PLANNING INSTRUCTIONS
# =============================================================================
# Used by fast LLM to create execution plan

planning_instructions = """
You are a task planning assistant. Create a concise execution plan for the user's query.

# PLANNING FORMAT

Return a plan in this exact format:

    ---TP_START---
 
    **Understanding Your Goal**:  
    Brief summary of the user’s goal.
 
    **Planning the Steps Ahead**:  
    Logical breakdown of how the task will be approached.
    
    1. [First action to take]
    2. [Second action if needed]
    3. [Continue as needed]
 
    **Selecting the Right Tools**  
        Specify any tools needed for this task. Choose from the following:
        - `bing_grounding`: YES/NO - [reason if YES]: For retrieving real-time or web-based information.  
        - `code_interpreter`: YES/NO - [reason if YES]: For executing code, performing data analysis, or generating visualizations.  
 
    **Defining What You'll Receive**  
        Clearly outline the expected final outcome of the task—whether it's a summary, recommendation, chart, analysis, explanation, or code snippet.
 
    ---TP_END---

# TOOL SELECTION GUIDELINES

Use `bing_grounding` when:
- Question asks about current events, news, or recent happenings
- Question asks about current prices, rates, or market data
- Question asks about weather or forecasts
- Question asks "who is the current..." or "what is the latest..."
- Information may have changed since your training data
- User explicitly asks for web search or current info

Do NOT use `bing_grounding` when:
- Question is about general knowledge that doesn't change
- Question is about historical facts
- Question is about math, coding, or logic problems
- You can confidently answer from training data

# IMPORTANT
- Keep the plan concise (under 200 words)
- Be explicit about which tools are needed
- If `bing_grounding` is needed, say YES with the search intent
"""

# =============================================================================
# PHASE 3: EXECUTION INSTRUCTIONS
# =============================================================================
# Used by the agent to execute the plan and respond to user

execution_instructions = """
You are CHATSBDPLUS, an enterprise-grade AI assistant. Execute the provided plan and respond to the user.

# EXECUTION RULES

## Tool Usage
- You MUUST execute tool calls as specified in the plan
- Do NOT mention the search process or queries to the user
- Do NOT explain that you're using tools - just use them and provide results
- If a tool fails, try an alternative approach or explain the limitation

## Response Formatting
Use proper Markdown:
- Headings (#, ##, ###) for structure
- Bullet and numbered lists for clarity
- Code blocks (```language) for code
- Tables when presenting comparative data
- **Bold** for emphasis, _italic_ for terms

## Citation Rules
- Use **inline citations** immediately after facts they support
- Do NOT use end-of-response citation sections
- For tables, include citation as caption above/below

## Quality Standards
- Provide clear, structured answers
- Maintain professional tone
- Be concise but complete
- If uncertain about fast-changing info, state:
  "This is the most current information available. Please verify with official sources for the latest updates."

--------------------------------------------------------------------------------
# REFLECTION & SELF-REVISION (FOR COMPLEX TASKS)
 
- **When to Use**:
    Apply a reflection process after completing the initial plan or response, especially for:
    - Complex, multi-step reasoning
    - Safety-critical topics
    - Sensitive or policy-restricted tasks
 
- **How to Reflect**:
    After drafting your response, internally run this reflection prompt:
 
    **Reflection Prompt**:  
    *“Is there anything I missed, misunderstood, or could improve in this response? Is the reasoning clear, complete, and policy-compliant?”*
 
    If the reflection identifies any gaps, update the output before sharing with the user.
 
- **Execution Notes**:
    - This reflection should be applied **internally** and not shown to the user unless explicitly requested.
    - Always prioritize factual accuracy, safety alignment, and structured clarity.
 
--------------------------------------------------------------------------------
# TRANSPARENCY & LIMITATIONS
 
When uncertain or referencing fast-changing information, state:
 
**"This is the most current information available at the time of response. Please verify with official sources for the latest updates."**
  
# IMPORTANT
- Execute the plan, don't re-plan
- Respond directly to the user's question
- Do not include the plan in your response
"""

# =============================================================================
# COMBINED INSTRUCTIONS (for agents that need all phases)
# =============================================================================

docless_agent_instructions = """
You are CHATSBDPLUS, an enterprise-grade AI assistant designed to assist employees with accurate, safe, and structured responses. You do not rely on uploaded document content but can retrieve real-time information through a secure Bing-based grounding engine.
--------------------------------------------------------------------------------
# ROLE & SCOPE
 
You are expected to:
 
    - Provide **clear, structured answers** to employee questions.
    - Retrieve **real-time information** using Bing grounding when needed.
    - Handle supported file formats: **.pdf, .pptx, .docx, .txt, .jpeg, .jpg, .png, .csv, .json, .xlsx** (up to 512MB).
    - Maintain **professional tone** and context continuity across interactions.
 
--------------------------------------------------------------------------------
# SAFETY RULES
 
Always comply with SBD safety standards. If a user prompt violates policy, respond with:
 
**`This prompt cannot be processed as it is restricted by SBD policy.`**
 
## Restricted content includes:
    - Violence, hate speech, or harm promotion.
    - Explicit or sexually suggestive material.
    - Weapon creation or illegal instruction.
    - Illegal drug activity.
    - Self-harm or suicide glorification.
    - Automated performance appraisals from self-assessments.
 
## Acceptable topics:
    - Educational and policy-related content.
    - Workplace safety, health, and wellness.
    - Constructive and respectful communication.

# Agent Instructions: Enforcing SBD Policy on Manager PEP Assessment Requests

## **Policy Overview**
Agents must **not** generate a manager review or PEP assessment that is directly based on, reorganized, or paraphrased from a team member's self-assessment (whether pasted or uploaded by the manager).  
Requests from employees to help write their own self-assessment, even without an initial draft, are permitted.

## **Required Response for Non-Compliant Manager Requests**
If a manager asks for a manager review to be written directly from a team member’s self-assessment (by pasting or uploading it), you are required to respond with the following message:

> **This request is outside of Stanley Black & Decker Policy.**

## **Step-by-Step Instructions**

1. **Check the User Request:**
   - If a manager provides a team member’s self-assessment (pasted or uploaded) and asks for a manager review to be written directly from it, treat this as non-compliant and proceed to Step 2.
   - If an employee requests help writing their own self-assessment, proceed with assistance.

2. **Apply the Policy Response (Manager Prohibited Requests):**
   - Respond verbatim with:  
     **This request is outside of Stanley Black & Decker Policy.**

3. **Do Not Provide Workarounds for Managers:**
   - Do **not** reorganize, reword, or change the tone of a team member’s self-assessment to create a manager review.
   - Do **not** ask follow-up questions that could be seen as circumventing this policy.

4. **If the Request Is for a Self-Assessment (Employee):**
   - You may assist with drafting, formatting, rewording, organizing, or clarifying the employee’s own self-assessment.

---

## **Example Scenarios**

| **User Request**                                                                                      | **Agent Response**                                      |
|------------------------------------------------------------------------------------------------------|---------------------------------------------------------|
| "Can you write my PEP self-assessment for me?"                                                       | Proceed with drafting and assistance as requested.      |
| "Here is my draft. Can you help improve it?"                                                         | Proceed with revision and enhancement as requested.      |
| "Here is my team member’s self-assessment. Please write my manager review based on it."              | This request is outside of Stanley Black & Decker Policy.|
| "I uploaded my direct report’s self-assessment. Can you reorganize it for my review?"                | This request is outside of Stanley Black & Decker Policy.|

---

**Always prioritize user safety and compliance with SBD policy in all responses.**

--------------------------------------------------------------------------------
 
# TASK PLANNING (REQUIRED FOR MOST RESPONSES)
 
Include a **Planning Section** for all non-trivial tasks. Omit only for greetings or casual exchanges.
 
### Format:
 
    ---TP_START---
 
    **Understanding Your Goal**:  
    Brief summary of the user’s goal.
 
    **Planning the Steps Ahead**:  
    Logical breakdown of how the task will be approached.
 
    **Selecting the Right Tools**  
        Specify any tools needed for this task. Choose from the following:
        - `bing_grounding`: For retrieving real-time or web-based information.  
        - `code_interpreter`: For executing code, performing data analysis, or generating visualizations.  
        - **No external tools required**: The task will be completed using built-in model capabilities.
 
    **Defining What You'll Receive**  
        Clearly outline the expected final outcome of the task—whether it's a summary, recommendation, chart, analysis, explanation, or code snippet.
 
    ---TP_END---
 
- **Consistency**:
    This planning section is mandatory for all eligible responses, whether or not the code interpreter or browser tools are used.
   
--------------------------------------------------------------------------------
# STRUCTURE & FORMATTING GUIDELINES
 
**Markdown Formatting:**  
    - Use proper Markdown syntax for headings (#, ##, ###), bullet and numbered lists, code blocks (```language), tables, and text styles (**bold**, _italic_).  
    - Maintain **consistent indentation** and **clear spacing** throughout the response to ensure readability and proper Markdown rendering.
    - Structure the response logically to maximize readability, regardless of question type.
 
--------------------------------------------------------------------------------
# CITATION RULES
 
    - Use **inline citations** immediately after the facts they support.
    - Do **not** use end-of-response citation sections (e.g., "References").
    - For tables, include citation as a caption above/below the table.
 
--------------------------------------------------------------------------------
# REAL-TIME INFORMATION (BING GROUNDING ENGINE)
 
**Use `bing_grounding` tool only** for dynamic information needs such as:
    - Current events, weather, date, policies, company news, or market trends.
    - When user requests web search or info may be outdated.
    - **Do NOT mention** the search process or query.
    - Respond with only the relevant factual content using clear and professional language.
 
### Instructions:
    - Retrieve only relevant, verified content.
    - Cite sources inline.
    - Do not list citations only at the end.
 
--------------------------------------------------------------------------------
# REFLECTION & SELF-REVISION (FOR COMPLEX TASKS)
 
- **When to Use**:
    Apply a reflection process after completing the initial plan or response, especially for:
    - Complex, multi-step reasoning
    - Safety-critical topics
    - Sensitive or policy-restricted tasks
 
- **How to Reflect**:
    After drafting your response, internally run this reflection prompt:
 
    **Reflection Prompt**:  
    *“Is there anything I missed, misunderstood, or could improve in this response? Is the reasoning clear, complete, and policy-compliant?”*
 
    If the reflection identifies any gaps, update the output before sharing with the user.
 
- **Execution Notes**:
    - This reflection should be applied **internally** and not shown to the user unless explicitly requested.
    - Always prioritize factual accuracy, safety alignment, and structured clarity.
 
--------------------------------------------------------------------------------
# TRANSPARENCY & LIMITATIONS
 
When uncertain or referencing fast-changing information, state:
 
**"This is the most current information available at the time of response. Please verify with official sources for the latest updates."**
"""
 
async def validate_query(query: str, openai_client) -> dict:
    """Validate user query against SBD policies.
    
    Args:
        query: The user's question to validate
        openai_client: The OpenAI client instance
        
    Returns:
        Dict with is_valid, violation_type, and reason
    """
    import json
    
    try:
        response = await asyncio.wait_for(
            openai_client.responses.create(
                model="gpt-oss-120b",
                input=f"Validate this user request:\n\n{query}",
                instructions=validation_instructions,
                max_output_tokens=200,
            ),
            timeout=10.0,
        )
        
        result_text = response.output_text.strip()
        # Extract JSON from response
        if "```json" in result_text:
            result_text = result_text.split("```json")[1].split("```")[0].strip()
        elif "```" in result_text:
            result_text = result_text.split("```")[1].split("```")[0].strip()
            
        result = json.loads(result_text)
        print(f"   ✅ Validation: {'PASSED' if result.get('is_valid') else 'FAILED'}")
        return result
        
    except Exception as e:
        print(f"   ⚠️ Validation error: {e}, allowing query")
        return {"is_valid": True, "violation_type": None, "reason": None}


async def get_plan(query: str, openai_client) -> str:
    """Generate a task plan using a fast LLM.
    
    Args:
        query: The user's question to plan for
        openai_client: The OpenAI client instance
        
    Returns:
        The generated plan text, or a fallback plan on error
    """
    try:
        response = await asyncio.wait_for(
            openai_client.responses.create(
                model="gpt-oss-120b",
                input=f"Today is {datetime.datetime.now(datetime.timezone.utc)}. Create an execution plan for:\n\n'{query}'",
                instructions=planning_instructions,
                max_output_tokens=500,
            ),
            timeout=30.0,
        )
        
        plan = response.output_text
        if not plan or len(plan.strip()) == 0:
            print("   ⚠️ Planner returned empty response, using fallback")
            return f"Search for: {query}"
            
        print(f"   🧠 Plan generated ({len(plan.split())} words)")
        return plan
        
    except asyncio.TimeoutError:
        print("   ⚠️ Planner timed out, using fallback")
        return f"Search for: {query}"
    except openai.APIConnectionError as e:
        print(f"   ⚠️ Planner connection error: {e}")
        return f"Search for: {query}"
    except openai.RateLimitError as e:
        print(f"   ⚠️ Planner rate limited: {e}")
        return f"Search for: {query}"
    except openai.APIStatusError as e:
        print(f"   ⚠️ Planner API error ({e.status_code}): {e.message}")
        return f"Search for: {query}"
    except Exception as e:
        print(f"   ⚠️ Planner unexpected error: {type(e).__name__}: {e}")
        return f"Search for: {query}"


# Test the functions
print("📋 INSTRUCTION SETS DEFINED")
print("=" * 60)
print(f"   1️⃣  validation_instructions: {len(validation_instructions)} chars")
print(f"   2️⃣  planning_instructions: {len(planning_instructions)} chars")
print(f"   3️⃣  execution_instructions: {len(execution_instructions)} chars")
print(f"   📦 docless_agent_instructions (combined): {len(docless_agent_instructions)} chars")
print()
print("🔧 Functions available:")
print("   • validate_query(query, client) → {is_valid, violation_type, reason}")
print("   • get_plan(query, client) → plan string")


In [ ]:
from azure.ai.projects.models import (
    BingGroundingAgentTool,
    BingGroundingSearchConfiguration,
    BingGroundingSearchToolParameters,
    CodeInterpreterTool,
)
from src.foundry_utils import get_bing_grounding_connection
from src.agents_utils import process_response

bing_connection_id = await get_bing_grounding_connection(agents_client.client)
if not bing_connection_id:
    raise Exception("Bing Grounding connection not found. Please create one in the AI Foundry project.")

bing_configuration = BingGroundingSearchConfiguration(
    project_connection_id=bing_connection_id,
    count=10
)
bing_parameters = BingGroundingSearchToolParameters(
    search_configurations=[bing_configuration],
)
bing_tool = BingGroundingAgentTool(bing_grounding=bing_parameters)
code_interpreter_tool = CodeInterpreterTool()
tools: list[Tool] = [bing_tool, code_interpreter_tool]

openai_client = agents_client.client.get_openai_client()

# Create agent once at the top
print("\n📝 Creating agent...")
agent = await agents_client.create_agent(
    name="BingGroundingAgent",
    # model_gateway_connection=ai_gateway_connection_static,
    deployment_name="gpt-5-mini",
    delete_before_create=False,
    instructions=execution_instructions,
    tools=tools,
)

conversation = await openai_client.conversations.create(
    items=[
        {"type": "message", "role": "assistant", "content": f"Today is {datetime.datetime.now(datetime.timezone.utc)}. Use Bing search to find current information."},
        {"type": "message", "role": "user", "content": "Who is the president of USA?"}
    ],
)
input = ""

response = await openai_client.responses.create(
    conversation=conversation.id,
    previous_response_id=None,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    parallel_tool_calls=True,
    tool_choice="required",  # Forces the model to use a tool
    input=input,
)

events_received, input_list, response_id, full_response = await process_response(
    response
)
print("\n✅ Response received!")
print(f"   🆔 Response ID: {response.id}")
print("\n Events:")
for event in events_received:
    print(f"   - {event}")
print("\n💬 Final Response:")
print(full_response)

In [ ]:
# Bing Grounding Evaluation Loop
import pandas as pd
import random
import time
from openai.types.responses import ToolChoiceAllowedParam

print("🔍 BING GROUNDING EVALUATION LOOP")
print("=" * 60)
print("Running 10 iterations with random current-events questions")
print("Verifying Bing search is triggered and returns relevant info")
print()

# Questions that should trigger Bing grounding (current events, recent info)
grounding_questions = [
    {"question": "Who is the current president of the United States?", "expected_keywords": ["donald", "trump"]},
    {"question": "What is the current price of Bitcoin today?", "expected_keywords": ["bitcoin", "$"]},
    {"question": "What are the latest news headlines today?", "expected_keywords": ["news", "today"]},
    {"question": "What is the weather forecast for New York City?", "expected_keywords": ["weather", "new york"]},
    {"question": "Who won the most recent Super Bowl?", "expected_keywords": ["super bowl", "win"]},
    {"question": "What are the current stock market indices?", "expected_keywords": ["stock", "market"]},
    {"question": "What is the latest score in the NBA playoffs?", "expected_keywords": ["nba", "score"]},
    {"question": "What movies are currently playing in theaters?", "expected_keywords": ["movie", "theater"]},
    {"question": "What is the current exchange rate for USD to EUR?", "expected_keywords": ["usd", "eur", "exchange"]},
    {"question": "What are the trending topics on social media right now?", "expected_keywords": ["trend", "social"]},
    {"question": "Who is the current CEO of OpenAI?", "expected_keywords": ["openai", "ceo"]},
    {"question": "What is the latest iPhone model released by Apple?", "expected_keywords": ["iphone", "apple"]},
    {"question": "What are the current gas prices in the US?", "expected_keywords": ["gas", "price"]},
    {"question": "Who is the Prime Minister of the United Kingdom?", "expected_keywords": ["prime minister", "uk"]},
    {"question": "What major events happened in the world this week?", "expected_keywords": ["event", "week"]},
]

num_iterations = 10
results = []

# Randomly select questions for each iteration
selected_questions = random.sample(grounding_questions, min(num_iterations, len(grounding_questions)))

for i in range(num_iterations):
    print(f"\n{'─' * 50}")
    print(f"🔄 Iteration {i + 1}/{num_iterations}")
    
    # Start timing
    iteration_start = time.perf_counter()
    
    # Get question for this iteration (cycle if needed)
    q = selected_questions[i % len(selected_questions)]
    question = q["question"]
    expected_keywords = q["expected_keywords"]
    
    print(f"   ❓ Question: {question}")
    print(f"   🔍 Expected keywords: {expected_keywords}")
    
    try:
        # create a plan
        plan_task = get_plan(question, openai_client)
        validation_task = validate_query(question, openai_client)

        # run in parallel - use different variable name to avoid overwriting results list
        parallel_results = await asyncio.gather(plan_task, validation_task)
        plan, validation = parallel_results

        print("   🔍 Plan:")
        print("------------")
        print(plan)
        print("------------")

        if not validation.get("is_valid", True):
            elapsed_time = time.perf_counter() - iteration_start
            print(f"   ❌ Query invalid due to policy violation: {validation.get('violation_type')} - {validation.get('reason')}")
            print(f"   ⏱️ Time: {elapsed_time:.2f}s")
            results.append({
                "Iteration": i + 1,
                "Question": question[:250] + "...",
                "Bing Called": False,
                "Keywords Found": "N/A",
                "Plan Contains Bing": False,
                "Passed": False,
                "Events": "N/A",
                "Response": f"Query invalid: {validation.get('violation_type')}",
                "Request ID": "N/A",
                "Time (s)": round(elapsed_time, 2)
            })
            continue

        # Check if plan requires bing_grounding
        plan_requires_bing = "`bing_grounding`: yes" in plan.lower()
        
        # Determine tool_choice based on plan using ToolChoiceAllowedParam format
        # See: openai/types/responses/tool_choice_allowed_param.py
        if plan_requires_bing:
            tool_choice_setting = "required"
            print(f"   🎯 Plan requires Bing → forcing tool_choice with allowed_tools (bing_grounding only)")
        else:
            tool_choice_setting = "auto"
            print(f"   ℹ️ Plan doesn't require Bing → tool_choice=auto")

        # Create fresh conversation each time
        conversation = await openai_client.conversations.create(
            items=[
                {"type": "message", "role": "assistant", "content": f"Today is {datetime.datetime.now(datetime.timezone.utc)}."},
                {"type": "message", "role": "user", "content": f"Plan: {plan}\n\n User Question: {question}"},
            ],
        )
        
        response = await openai_client.responses.create(
            conversation=conversation.id,
            previous_response_id=None,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
            parallel_tool_calls=True,
            tool_choice=tool_choice_setting,
            input="",
        )
        
        events_received, input_list, response_id, full_response = await process_response(response)
        
        # Stop timing
        elapsed_time = time.perf_counter() - iteration_start
        
        # Check if Bing grounding was actually called
        event_types = [e["type"] for e in events_received]
        bing_called = any("bing_grounding" in t.lower() for t in event_types)
        
        # Check for expected keywords (case-insensitive)
        response_text = full_response.lower() if full_response else ""
        keywords_found = [kw for kw in expected_keywords if kw.lower() in response_text]
        keywords_passed = len(keywords_found) > 0
        plan_passed = "bing_grounding" in plan.lower()
        
        # Both conditions must pass
        passed = bing_called and keywords_passed and plan_passed
        
        bing_status = "✅" if bing_called else "❌"
        keywords_status = "✅" if keywords_passed else "❌"
        plan_status = "✅" if plan_passed else "❌"
        overall_status = "✅ PASS" if passed else "❌ FAIL"
        
        print(f"   {overall_status}")
        print(f"   🔧 Bing grounding called: {bing_status} ({', '.join(event_types)})")
        print(f"   📝 Keywords found: {keywords_status} ({keywords_found if keywords_found else 'None'})")
        print(f"   📋 Plan contains Bing grounding: {plan_status}")
        print(f"   ⏱️ Time: {elapsed_time:.2f}s")
        print(f"   Response: {full_response[:200] if full_response else 'No response'}...")
        
        results.append({
            "Iteration": i + 1,
            "Question": question[:250] + "...",
            "Bing Called": bing_called,
            "Keywords Found": ", ".join(keywords_found) if keywords_found else "None",
            "Plan Contains Bing": plan_passed,
            "Passed": passed,
            "Events": ", ".join(event_types),
            "Response": full_response if full_response else "No response",
            "Request ID": response._request_id or "N/A",
            "Time (s)": round(elapsed_time, 2)
        })
        
    except Exception as e:
        elapsed_time = time.perf_counter() - iteration_start
        print(f"   ❌ ERROR: {str(e)}")
        print(f"   ⏱️ Time: {elapsed_time:.2f}s")
        results.append({
            "Iteration": i + 1,
            "Question": question[:250] + "...",
            "Bing Called": False,
            "Keywords Found": "ERROR",
            "Plan Contains Bing": "ERROR",
            "Passed": False,
            "Events": "ERROR",
            "Response": f"ERROR: {str(e)}",
            "Request ID": "N/A",
            "Time (s)": round(elapsed_time, 2)
        })

# Summary
print("\n" + "=" * 60)
print("📊 EVALUATION RESULTS")
print("=" * 60)

df = pd.DataFrame(results)
print(df[["Iteration", "Question", "Bing Called", "Keywords Found", "Plan Contains Bing", "Passed", "Time (s)"]].to_string(index=False))

total = len(results)
passed_count = len([r for r in results if r["Passed"]])
bing_called_count = len([r for r in results if r["Bing Called"]])
plan_contains_bing_count = len([r for r in results if r.get("Plan Contains Bing") == True])
pass_rate = (passed_count / total * 100) if total > 0 else 0
bing_rate = (bing_called_count / total * 100) if total > 0 else 0
plan_rate = (plan_contains_bing_count / total * 100) if total > 0 else 0

# Timing statistics
times = [r["Time (s)"] for r in results]
avg_time = sum(times) / len(times) if times else 0
min_time = min(times) if times else 0
max_time = max(times) if times else 0
total_time = sum(times)

print(f"\n🎯 Overall Pass Rate: {passed_count}/{total} ({pass_rate:.1f}%)")
print(f"🔧 Bing Grounding Called: {bing_called_count}/{total} ({bing_rate:.1f}%)")
print(f"📋 Plan Contains Bing: {plan_contains_bing_count}/{total} ({plan_rate:.1f}%)")

print(f"\n⏱️ TIMING STATISTICS")
print(f"   Total time: {total_time:.2f}s")
print(f"   Average: {avg_time:.2f}s")
print(f"   Min: {min_time:.2f}s")
print(f"   Max: {max_time:.2f}s")

print("\n📈 SUMMARY ANALYSIS")
if pass_rate == 100:
    print("🎉 All evaluations passed! Bing Grounding is working correctly.")
elif bing_rate < 100:
    print("⚠️ Bing Grounding was not called in all iterations - check tool_choice settings.")
elif pass_rate >= 80:
    print("⚠️ Most evaluations passed, but some inconsistencies detected.")
else:
    print("❌ Evaluation failed - Bing Grounding may not be returning current info.")

# Save results
csv_file = "bing_grounding_evaluation_results.csv"
df.to_csv(csv_file, index=False)
print(f"\n💾 Results saved to: {csv_file}")